In [9]:
#===============================================================================
import pandas as pd
import feather as fd
import numpy as np
from datetime import datetime


# Tratamiento de archivos
# ===============================================================================
import feather
import datetime as dt

# Base de datos 
# ===============================================================================
import pandas as pd 
import  pyodbc
# Graficos
# ===============================================================================
import plotly.graph_objects as go
import plotly.express as px
from scipy.spatial import distance
#------------------------------------------------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')


In [11]:
# Nos conectamos a la base de datos y generamos un archivo feather:
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=dbtstanalytics.database.windows.net;DATABASE=tstanalytics;UID=analyticsreader;PWD=@q$eQX89Xp7yUEm')
# Creamos la consulta:
df = pd.read_sql_query('SELECT * FROM Output_EstadoDeResultados', conn ) 
# De dataframe pasamos a formato feather:
df.to_feather('df.feather')
df1=pd.read_feather('df.feather')
df2=pd.read_feather('df.feather')
df1.head(2)

,NroAsto,Fecha,EmpresaCod,EmpresaNombre,Monto,NroComp,ComprobanteTipo,CuentaNro,CuentaNombre,Leyenda,...,CodNegocio,IdProy,IdCli,Cliente,Uneg,Fimp,Clase,IdLider,IdCoord,IdResp
0,5678,2021-03-29,1,INTEROPTICS S.A.,-3600.0,105,FA,4210201,FLETES Y ACARREOS,ARIAS ERNESTO LUIS,...,IPL20-001,36.0,26,NSS SA,DESP,2021-06-27,NOF,87,87,87
1,1158,2021-04-06,5,INTRANET S.A.,-2780.0,3549,RND,4210214,RENDICIONES,RENDICIONES,...,ICS21-001,74.0,51,INTEROPTICS S.A.,INET,2021-04-15,FAC,113,127,127


## Exploración datos-Obras:

#### Alerta: Fecha imp
- Fecha de imputación por obra, ver si pasan los 90 días.

In [3]:
Dto_Obra = df2[['IdProy','ObraNro', 'Fecha', 'Fimp', 'TipoMov', 'Monto', 'ObraNombre']]
Dto_Obra['Fecha'] = pd.to_datetime(Dto_Obra['Fecha'])
Dto_Obra['Fimp'] = pd.to_datetime(Dto_Obra['Fimp']) 

In [4]:
Dto_Obra['Diferencia'] = Dto_Obra['Fimp'] - Dto_Obra['Fecha']
Dto_Obra['Diferencia'] = Dto_Obra['Diferencia'].dt.days
Dto_Obra['Alerta'] = np.where(Dto_Obra['Diferencia'] > 90, 'Alerta', 'Bien')
alertas = Dto_Obra[Dto_Obra['Alerta'] == 'Alerta']
alertas.head(3)

,IdProy,ObraNro,Fecha,Fimp,TipoMov,Monto,ObraNombre,Diferencia,Alerta
32535,4.0,244394,2021-08-31,2022-01-01,GTO,-25000.00,IMPUTACIÓN TRANSITORIA DE GASTOS UN MTTO: SEPT...,123,Alerta
32536,4.0,244394,2021-08-31,2022-01-01,GTO,-381469.37,IMPUTACIÓN TRANSITORIA DE GASTOS UN MTTO: SEPT...,123,Alerta
32537,4.0,244394,2021-09-01,2022-01-01,GTO,3305.79,IMPUTACIÓN TRANSITORIA DE GASTOS UN MTTO: SEPT...,122,Alerta


In [5]:
alertas.IdProy.unique()

array([ 4, 28], dtype=int64)

In [6]:
cantidad_alertas = alertas[alertas['Alerta'] == 'Alerta'].shape[0]
print("Cantidad de veces que hay alerta en el DataFrame:", cantidad_alertas)

Cantidad de veces que hay alerta en el DataFrame: 180


In [7]:
               # Identificamos el proyecto con mas alertas

Alerta_proy = alertas[alertas['Alerta'] == 'Alerta']['IdProy'].value_counts()
Alerta_proy = Alerta_proy.idxmax()
# Obras de el proyecto que se imputo a destiempo:
obras_proyecto_alerta = alertas[(alertas['Alerta'] == 'Alerta') & (alertas['IdProy'] == Alerta_proy)]['ObraNro']
# Imprimimos
print("El proyecto con la mayoría de obras con imputación a destiempo:", Alerta_proy)
print("Obras del proyecto con la mayoría de obras en alerta:" )
print(obras_proyecto_alerta.unique())
print("Las veces imputadas mal: " + str(obras_proyecto_alerta.count()))


El proyecto con la mayoría de obras con imputación a destiempo: 4.0
Obras del proyecto con la mayoría de obras en alerta:
['244394' '244395']
Las veces imputadas mal: 150


#### Margen Bruto por Obra:

##### Análisis:

In [55]:
def calcular_margen_bruto(df):
    
    # Se filtran los datos por ventas:
    df_ventas = df[df['TipoMov'] == 'VTA']
    ventas_sum = df_ventas.groupby('ObraNro')['Monto'].sum().reset_index()
    umbral_ventas = 0.001 #Valor minimo de ventas por  el monto de ventas es menor que el de gasto
    
    ## float
    df_ventas['Monto'] = df_ventas['Monto'].astype(float) 
    df_ventas['Monto'] = df_ventas['Monto'].abs()
    
    # Sumamos gastos + actuvos
    df_gto_act = df[(df['TipoMov'] == 'GTO') | (df['TipoMov'] == 'ACT')]
    gto_act_sum = df_gto_act.groupby('ObraNro')['Monto'].sum().reset_index()  
    
    #Montos abs de gst y act ya que no son valores todos negativos
    df_gto_act['Monto'] = df_gto_act['Monto'].astype(float) 
    df_gto_act['Monto'] = df_gto_act['Monto'].abs()

    # Datos de Vta,Gto y Act en funcion de la obra:
    df_merge = ventas_sum.merge(gto_act_sum, on='ObraNro', how='inner')

    # Filtramos datos###### Vemos como podemos mejorar esto:
    df_merge_filtered = df_merge[df_merge['Monto_x'] > umbral_ventas]

    # Calculamos el margen bruto
    margen_bruto = (df_merge_filtered['Monto_x'] + df_merge_filtered['Monto_y']) / df_merge_filtered['Monto_x']
    
    # Dataframe final:
    df_resultado = pd.DataFrame({
     'Obra': df_merge_filtered['ObraNro'],
     'Gastos_Activos': df_merge_filtered['Monto_y'],
     'Ventas': df_merge_filtered['Monto_x'],
     'MargenBruto': margen_bruto
    })

    # Calculamos el porcentaje
    df_resultado['%MBruto'] = (df_resultado['MargenBruto'] * 100).round(2)
    
    return df_resultado

df_resultado = calcular_margen_bruto(Dto_Obra)
df_resultado.head(3)

# De 0 a 1 ganancia obtenida menor a las ventas
# Mayor a 1, Ganancia mayor a las ventas.

,Obra,Gastos_Activos,Ventas,MargenBruto,%MBruto
0,230858,-244163.58,5762854.0,0.957631,95.76
1,231273,-233051.26,5485046.0,0.957512,95.75
2,240052,-41947.40,366214.0,0.885457,88.55


##### Graficos:
- Regresión Lineal:

In [16]:
# trendline='ols' traza una línea de regresión lineal para mostrar la tendencia en los datos.
fig = px.scatter(df_resultado, x='Ventas', y='Gastos_Activos', hover_data=['Obra'], trendline='ols') 
fig.update_traces(marker=dict(size=10))  
fig.update_layout(title='Gráfico de dispersión: Ventas vs. Gastos_Activos')  
fig.update_xaxes(title='Ventas')  
fig.update_yaxes(title='Gastos')  
fig.show()

In [48]:
#Valores de las columnas 'Ventas' y 'Gastos_Activos'
X = df_resultado[['Ventas', 'Gastos_Activos']].values
#Distancias entre los puntos de datos utilizando la distancia euclidiana
dist_matrix = distance.cdist(X, X, 'euclidean')
threshold = 21500000

#Definimos el umbral
#Generamos el clusters segun el umbral:
clusters = []
assigned = [False] * len(X)

for i in range(len(X)):
    if not assigned[i]:
        cluster = [i]
        for j in range(i + 1, len(X)):
            if not assigned[j] and dist_matrix[i, j] <= threshold:
                cluster.append(j)
                assigned[j] = True
        clusters.append(cluster)

# Reseteamos el índice del DataFrame        
df_resultado.reset_index(drop=True, inplace=True)

#Asignación de los numeros de cluster que tenemos en el dataframe:
df_resultado['Cluster'] = -1

for i, cluster in enumerate(clusters):
    df_resultado.loc[cluster, 'Cluster'] = i
# Creamos del grafico:
fig = go.Figure()

# Agregamos los puntos de datos al gráfico
fig.add_trace(go.Scatter(
    x=df_resultado['Ventas'],
    y=df_resultado['Gastos_Activos'],
    mode='markers',
    marker=dict(color=df_resultado['Cluster'], colorscale='Viridis', size=10),
    hovertemplate='Obra: %{text}<br>Cluster: %{marker.color}<extra></extra>',
    text=df_resultado['Obra']
))

# Personaliza el diseño del gráfico
fig.update_layout(
    title='Gráfico de dispersión con Clustering',
    xaxis=dict(title='Ventas'),
    yaxis=dict(title='Gastos_Activos')
)

# Mostrar el gráfico
fig.show()


In [49]:
#Cantidad de clusters que generamos:
num_clusters = len(clusters)
print("Cantidad de Clusters:", num_clusters)
#Imprimimos obras por clusters: 
for i in range(num_clusters):
    cluster_label = "Cluster " + str(i)
    obras = df_resultado[df_resultado['Cluster'] == i]['Obra'].tolist()
    print(cluster_label + ":")
    print(obras)
    print()

Cantidad de Clusters: 8
Cluster 0:
['230858', '231273', '240052', '241289', '241537', '242584', '242863', '242864', '242865', '243127', '243128', '243215', '243216', '243249', '243331', '243332', '243460', '243472', '243476', '243510', '243529', '243530', '243531', '243532', '243534', '243535', '243536', '243537', '243547', '243552', '243553', '243555', '243556', '243560', '243563', '243566', '243580', '243595', '243608', '243625', '243626', '243627', '243628', '243629', '243630', '243633', '243645', '243648', '243656', '243657', '243659', '243667', '243668', '243669', '243673', '243676', '243684', '243685', '243688', '243689', '243693', '243696', '243697', '243703', '243708', '243711', '243712', '243713', '243723', '243734', '243735', '243736', '243737', '243738', '243739', '243740', '243751', '243752', '243753', '243754', '243755', '243756', '243757', '243758', '243759', '243760', '243763', '243764', '243765', '243766', '243768', '243774', '243775', '243777', '243778', '243779', '243

In [51]:
####### Obras cluster-Margen bruto
for i in range(num_clusters):
    
    cluster_label = "Cluster " + str(i)
    obras = df_resultado[df_resultado['Cluster'] == i]['Obra'].tolist()
    cluster_margin = df_resultado[df_resultado['Cluster'] == i]['MargenBruto']
    
    # Imprime las obras del cluster
    print(cluster_label + ":")
    print(obras)
    print()
    
    #EDescriptivas de margen bruto
    print("Estadísticas del margen bruto:")
    print("Promedio:", cluster_margin.mean())
    print("Mediana:", cluster_margin.median())
    print("Desviación estándar:", cluster_margin.std())
    print()
    
    #Grafico comparativo/margen bruto
    fig = go.Figure()
    fig.add_trace(go.Box(y=cluster_margin, name='Margen Bruto'))
    fig.update_layout(title=cluster_label + ': Distribución del Margen Bruto',
                      yaxis=dict(title='Margen Bruto'))
    fig.show()


Cluster 0:
['230858', '231273', '240052', '241289', '241537', '242584', '242863', '242864', '242865', '243127', '243128', '243215', '243216', '243249', '243331', '243332', '243460', '243472', '243476', '243510', '243529', '243530', '243531', '243532', '243534', '243535', '243536', '243537', '243547', '243552', '243553', '243555', '243556', '243560', '243563', '243566', '243580', '243595', '243608', '243625', '243626', '243627', '243628', '243629', '243630', '243633', '243645', '243648', '243656', '243657', '243659', '243667', '243668', '243669', '243673', '243676', '243684', '243685', '243688', '243689', '243693', '243696', '243697', '243703', '243708', '243711', '243712', '243713', '243723', '243734', '243735', '243736', '243737', '243738', '243739', '243740', '243751', '243752', '243753', '243754', '243755', '243756', '243757', '243758', '243759', '243760', '243763', '243764', '243765', '243766', '243768', '243774', '243775', '243777', '243778', '243779', '243780', '243781', '243782'

Cluster 1:
['241872', '244146', '244244', '244306', '244372', '244460', '244522', '244588', '244624', '244694', '244735', '244828', '244858', '244911', '245034', '245469', '245539', '245605', '245679', '245771']

Estadísticas del margen bruto:
Promedio: 0.32051285915320826
Mediana: 0.2439653439157914
Desviación estándar: 0.2286716982774021



Cluster 2:
['243679', '243762', '244537', '244695', '244830', '244898', '244915', '244993', '245076', '245157', '245396', '245470', '245540', '245542', '245558', '245606', '245608', '245609', '245641', '245657', '245672', '245677', '245680', '245681', '245682', '245730', '245774', '245844']

Estadísticas del margen bruto:
Promedio: 0.346218794527215
Mediana: 0.4075087950212988
Desviación estándar: 0.3130210011683776



Cluster 3:
['243841', '243952', '244052', '245457', '245543', '245585', '245758']

Estadísticas del margen bruto:
Promedio: -0.1977102009984346
Mediana: -0.42878565578183714
Desviación estándar: 0.3395571495958112



Cluster 4:
['244990', '245070', '245153', '245240', '245289', '245588']

Estadísticas del margen bruto:
Promedio: 0.3541205781549322
Mediana: 0.36413146583510037
Desviación estándar: 0.09790051547634528



Cluster 5:
['245027', '245381']

Estadísticas del margen bruto:
Promedio: 0.4872281449421043
Mediana: 0.4872281449421043
Desviación estándar: 0.06625481121011288



Cluster 6:
['245373']

Estadísticas del margen bruto:
Promedio: 0.7235915386124129
Mediana: 0.7235915386124129
Desviación estándar: nan



Cluster 7:
['245456', '245536', '245671', '245759']

Estadísticas del margen bruto:
Promedio: 0.3267820289867315
Mediana: 0.32729338564310806
Desviación estándar: 0.05752755441171984



In [52]:
# Analizar esto:
any(df_resultado['%MBruto']<0) # esto es raro, por que sucede?? 

True

In [53]:
# Obtener el índice de la obra con el margen bruto máximo
indice_max_margen_bruto = df_resultado['MargenBruto'].idxmax()
# Obtener el índice de la obra con el margen bruto mínimo
indice_min_margen_bruto = df_resultado['MargenBruto'].idxmin()
# Obtener la información completa de la obra con el margen bruto máximo
obra_max_margen_bruto = df_resultado.loc[indice_max_margen_bruto]
# Obtener la información completa de la obra con el margen bruto mínimo
obra_min_margen_bruto = df_resultado.loc[indice_min_margen_bruto]

In [54]:
# Imprimir la obra con el margen bruto máximo
print("Obra con mayor margen bruto:")
print("Obra:", obra_max_margen_bruto['Obra'])
print("Gastos_Activos:", obra_max_margen_bruto['Gastos_Activos'])
print("Ventas:", obra_max_margen_bruto['Ventas'])
print("MargenBruto:", obra_max_margen_bruto['MargenBruto'])
print("%MBruto:", obra_max_margen_bruto['%MBruto'])
print()

Obra con mayor margen bruto:
Obra: 244397
Gastos_Activos: 0.020000000340587576
Ventas: 0.02
MargenBruto: 2.000000017029379
%MBruto: 200.0

